In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [20]:

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilepreprocess
from tensorflow.keras.applications.vgg19 import preprocess_input as vggpreprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnetpreprocess
from keras.preprocessing import image
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from tensorflow.keras import mixed_precision
from tensorflow.keras.layers import BatchNormalization

mixed_precision.set_global_policy('mixed_float16')

generator = ImageDataGenerator(rescale = 1./255,
                               shear_range = 0.2,
                                zoom_range = 0.2,
                               validation_split = 0.20)

train_set = generator.flow_from_directory('../input/exml-prelims-task-2/train/train', batch_size = 32, target_size = (224, 224),
                                         color_mode = 'rgb', class_mode = 'categorical', subset = 'training')

valid_set = generator.flow_from_directory('../input/exml-prelims-task-2/train/train', batch_size = 32, target_size = (224, 224),
                                          color_mode = 'rgb', class_mode = 'categorical', subset = 'validation')

In [21]:
InceptionV3_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [22]:
for layer in InceptionV3_model.layers:
    layer.trainable = False

x = InceptionV3_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)

output = Dense(units=8, activation='softmax', dtype='float32')(x)
model = Model(InceptionV3_model.input, output)


# model.summary()

In [23]:
opt = optimizers.Adam()
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [24]:
history = model.fit(train_set, validation_data=valid_set, epochs=8,steps_per_epoch=5,validation_steps=32, verbose=1)

In [25]:
model.save('task2model.h5')

print(history.history['val_accuracy'])
print(history.history['val_loss'])

In [ ]:
from skimage import io
import numpy as np
import cv2
from skimage.transform import resize
import os
res = {}
tally = 0

for image in os.listdir("../input/exml-prelims-task-2/test/test/"):
    img = cv2.imread(f"../input/exml-prelims-task-2/test/test/{image}")
    img = cv2.resize(img, (224,224))
    img = np.expand_dims(img, axis = 0)

    predicted = model.predict(img).reshape(-1)
    print(predicted)
    company = np.argmax(predicted) + 1
    
    res[image.replace(".png", "")] = company
    print(f"{tally} images predicted")
    
    tally += 1

# image = cv2.imread("../input/exml-prelims-task-2/test/test/xtjEJZKFnZ_565719.png")
# plt.imshow(image)
# image = cv2.resize(image, (100,100))
# image = np.expand_dims(image, axis = 0)

# predicted = model.predict(image).reshape(-1)
# print(predicted)
# company = np.argmax(predicted)+1

# print(company)

In [ ]:
import pandas as pd
df = pd.DataFrame(res.items(), columns=['id', 'company'])
df.to_csv("submission2fortask2.csv", index=False)